In [3]:
# load style
from ROOT import gROOT

path = '../../'
# path = '/eos/user/y/youyou/'
gROOT.SetMacroPath(path+"atlasstyle/")
gROOT.LoadMacro("AtlasLabels.C")
gROOT.LoadMacro("AtlasStyle.C")
gROOT.LoadMacro("AtlasUtils.C")
from ROOT import SetAtlasStyle, ATLASLabel, myText, myMarkerText
SetAtlasStyle()

Welcome to JupyROOT 6.26/02

Applying ATLAS style settings...



In [225]:
# load lib
import ROOT
import os, sys
import numpy as np
from array import array
import pandas as pd

In [5]:
# load files

# standard model tree
sm_file = path + 'EFT_MINI_TREES/SM.root'
sm_data = ROOT.TFile(sm_file,'READ')
sm_tree = sm_data.vbswy_tree

# int and quad trees
eft_path = path + 'EFT_MINI_TREES/'
files = [(eft_path+i) for i in os.listdir(eft_path)]
files.sort()
files.remove(sm_file)
int_files = [i for i in files if 'INT' in i]
quad_files = [i for i in files if 'QUAD' in i]

# names of all operators we used
operators = [i.split('/')[-1].split('_')[0] for i in int_files]
operators.index('T7')

12

In [6]:
# list of branches we interested in 
names = [g.GetName() for g in sm_tree.GetListOfBranches()]
truth_branch = [i for i in names if 'truth' in i]
truth_branch = list(dict.fromkeys(truth_branch))
truth_branch.sort()

is_reconst = lambda i: not (('truth' in i) or ('weight' in i) or ('pass_baseline' in i))
reconst_branch = [i for i in names if is_reconst(i)]
reconst_branch = list(dict.fromkeys(reconst_branch))
reconst_branch.sort()

reconst_branch[1:]

['gam_e',
 'gam_eta',
 'gam_phi',
 'gam_pt',
 'j0_e',
 'j0_eta',
 'j0_phi',
 'j0_pt',
 'j1_e',
 'j1_eta',
 'j1_phi',
 'j1_pt',
 'jj_dphi_signed',
 'jj_drap',
 'jj_m',
 'jj_pt',
 'lep_e',
 'lep_eta',
 'lep_phi',
 'lep_pt',
 'w_e',
 'w_eta',
 'w_mt',
 'w_phi',
 'w_pt',
 'wy_m',
 'wy_phi',
 'wy_pt']

In [7]:
# def some helper functions
def generate_output_path(operator,add='', format = 'pdf'):
    folder = 'rebin_reconstructed/'
    folder = './output_'+format+'/'+folder
    if not os.path.exists(folder):
        os.mkdir(folder)
    return (folder+operator+add+'.'+format)

generate_output_path('T0')

'./output_pdf/rebin_reconstructed/T0.pdf'

In [8]:
def pre_selection(tree):
    entryList = ROOT.TEntryList("entryListName", "Title of the entry list")
    for entry in tree:
        gam = ROOT.TLorentzVector()
        w_boson = ROOT.TLorentzVector()
        jet0 = ROOT.TLorentzVector()
        jet1 = ROOT.TLorentzVector()
        wy = ROOT.TLorentzVector()

        gam.SetPtEtaPhiE(entry.gam_pt, entry.gam_eta, entry.gam_phi, entry.gam_e)
        w_boson.SetPtEtaPhiE(entry.w_pt, entry.w_eta, entry.w_phi, entry.w_e)
        jet0.SetPtEtaPhiE(entry.j0_pt, entry.j0_eta, entry.j0_phi, entry.j0_e)
        jet1.SetPtEtaPhiE(entry.j1_pt, entry.j1_eta, entry.j1_phi, entry.j1_e)
        wy = gam+w_boson
        # i didn't add it as a branch
        wy_xi = abs(wy.Rapidity()-0.5*(jet0.Rapidity()+jet1.Rapidity())) / entry.jj_drap

        selection = (int(entry.pass_baseline_e) | int(entry.pass_baseline_mu)) and (entry.jj_m > 0.5000) and (entry.jj_drap > 2) and (wy_xi < 0.35)
        if selection:
            entryList.Enter(tree.GetReadEntry())

    tree.SetEntryList(entryList)
    
    df = pd.DataFrame(columns=reconst_branch+['weight'])
    i = 0
    tmp = tree.GetEntryNumber(i)
    while tmp != -1:
        tree.GetEntry(tmp)
        list_tmp = []
        for r in reconst_branch:
            list_tmp += [getattr(tree, r)]
        list_tmp += [tree.weight]
        df.loc[i] = list_tmp
        i += 1
        tmp = tree.GetEntryNumber(i)

    return df   # tree, entryList


In [156]:
def generate_dr_dphi(df):
    df = df.copy()

    def mpi_pi(angle):
        while (angle >= np.pi):
            angle -= 2* np.pi
        while (angle < -1* np.pi):
             angle += 2* np.pi

        return angle

    dr_lepy = []
    dr_wy = []
    dphi_lepy = []
    dphi_wy = []
    for i in range(len(df)):
        gamma = ROOT.TLorentzVector()
        lepton = ROOT.TLorentzVector()
        w_boson = ROOT.TLorentzVector()

        gamma.SetPtEtaPhiE(df['gam_pt'][i],df['gam_eta'][i],df['gam_phi'][i],df['gam_e'][i])
        lepton.SetPtEtaPhiE(df['lep_pt'][i],df['lep_eta'][i],df['lep_phi'][i],df['lep_e'][i])
        w_boson.SetPtEtaPhiE(df['w_pt'][i],df['w_eta'][i],df['w_phi'][i],df['w_e'][i])

        dr_lepy += [lepton.DeltaR(gamma)]
        dr_wy += [w_boson.DeltaR(gamma)]

        if lepton.Rapidity() > gamma.Rapidity():
            lepy_phi_forward = lepton.Phi()
            lepy_phi_backward = gamma.Phi()
        else:
            lepy_phi_forward = gamma.Phi()
            lepy_phi_backward = lepton.Phi()
        dphi_lepy += [mpi_pi(lepy_phi_forward - lepy_phi_backward)]

        if w_boson.Rapidity() > gamma.Rapidity():
            wy_phi_forward = w_boson.Phi()
            wy_phi_backward = gamma.Phi()
        else:
            wy_phi_forward = gamma.Phi()
            wy_phi_backward = w_boson.Phi()
        dphi_wy += [mpi_pi(wy_phi_forward - wy_phi_backward)]

    df['dphi_lepy'] = dphi_lepy
    df['dphi_wy'] = dphi_wy
    df['dr_lepy'] = dr_lepy
    df['dr_wy'] = dr_wy
    
    return df    

In [157]:
sm_df = pre_selection(sm_tree)
sm_df = generate_dr_dphi(sm_df)

sm_df



,dsid,gam_e,gam_eta,gam_phi,gam_pt,j0_e,j0_eta,j0_phi,j0_pt,j1_e,...,w_phi,w_pt,wy_m,wy_phi,wy_pt,weight,dphi_lepy,dphi_wy,dr_lepy,dr_wy
0,363270.0,187.574789,2.042001,-0.503302,47.876445,829.435168,-1.997017,2.031665,221.087766,369.512384,...,-0.263019,107.960786,176.138268,-0.336648,154.881409,0.064699,-0.240210,-0.240283,2.070275,2.070284
1,363270.0,716.515885,-1.210340,2.881138,392.318937,2107.599090,3.068873,0.048355,195.462002,2157.751512,...,-0.059591,51.160642,325.896852,2.851317,342.339108,0.079482,-2.940507,-2.940729,3.138830,3.139038
2,363270.0,76.092459,0.221988,-2.728928,74.255336,409.161278,-2.048432,-0.372453,103.671930,98.808764,...,2.673311,38.194998,46.134245,-3.019382,102.872383,0.058395,0.880951,0.880946,0.894036,0.894031
3,363270.0,28.663095,0.672188,-0.054652,23.217365,1220.408780,-2.114067,2.516062,290.448906,956.535871,...,0.161921,209.252560,48.989156,0.140413,231.981212,0.074781,0.216600,0.216573,0.691692,0.691684
4,363270.0,27.678104,0.501819,2.357176,24.524812,945.942935,3.139888,1.492426,81.739844,336.349470,...,-2.981528,47.384492,73.421696,2.990389,64.877674,0.056811,-0.944477,-0.944481,1.969929,1.969932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19944,363272.0,73.200317,1.333682,-2.003242,36.072645,192.523441,-1.022055,-0.004206,122.415944,363.627920,...,1.695181,41.136863,75.141904,2.762000,21.771143,0.091927,2.584761,2.584763,2.605220,2.605222
19945,363272.0,156.622332,-1.909200,1.346155,45.424844,120.358292,-0.219015,-1.085886,117.030790,898.554284,...,2.376111,32.202403,101.735515,1.765037,67.872401,0.086812,1.030489,1.029957,2.318521,2.318285
19946,363272.0,105.070789,-0.993770,2.689175,68.414883,534.734661,-1.341179,-0.147029,261.314715,491.413472,...,-2.860134,53.229632,81.681235,3.008171,113.677272,0.093809,0.734089,0.733877,1.316264,1.316145
19947,363272.0,28.963492,0.117419,0.791893,28.764971,2425.672927,-1.740736,-2.701662,825.331489,2164.045073,...,0.158732,70.835859,54.745198,0.337810,95.553110,0.039691,-0.633298,-0.633161,1.182595,1.182522


In [158]:
def generate_df(operator):
    i = operators.index(operator)
    int_data = ROOT.TFile(int_files[i],'READ')
    int_tree = int_data.vbswy_tree
    quad_data = ROOT.TFile(quad_files[i], 'READ')
    quad_tree = quad_data.vbswy_tree

    # load values of coupling
    c1, c2, c3 = 1,10,100
    if operator[0] == 'T':
        c1, c2, c3 = 0.1,1,10

    # apply pre selection
    int_df = pre_selection(int_tree)
    quad_df = pre_selection(quad_tree)
    
    int_data.Close()
    quad_data.Close()

    # add drap and dphi
    int_df = generate_dr_dphi(int_df)
    quad_df = generate_dr_dphi(quad_df)

    sm_df['c1_w'] = sm_df['weight']
    sm_df['c2_w'] = sm_df['weight']
    sm_df['c3_w'] = sm_df['weight']
    int_df['c1_w'] = int_df['weight']*c1/2.4
    int_df['c2_w'] = int_df['weight']*c2/2.4
    int_df['c3_w'] = int_df['weight']*c3/2.4
    quad_df['c1_w'] = quad_df['weight']*(c1/2.4)**2
    quad_df['c2_w'] = quad_df['weight']*(c2/2.4)**2
    quad_df['c3_w'] = quad_df['weight']*(c3/2.4)**2

    df = pd.concat([sm_df, int_df, quad_df], keys=["s", "i", "q"])
    return df
    

In [159]:

l1 = ['gam_e','lep_pt','j0_pt','j1_pt','w_mt','wy_m','wy_pt']
l2 = ['jj_m','jj_pt','jj_dphi_signed']
branches = l2+l1

operators_t = [i for i in operators if i[0]=='T']
operators_m = [i for i in operators if i[0]=='M']
dfs_t = [generate_df(i) for i in operators_t]
dfs_m = [generate_df(i) for i in operators_m]


In [163]:
unit_dict= {'gam_e':    "E_{#gamma} [GeV]",
        'lep_pt':       "Lepton p_{T} [GeV/c] ",
        'j0_pt':        "Leading jet p_{T}  [GeV/c] ",
        'j1_pt':        "Second Leading jet p_{T} [GeV/c] ",
        'jj_m':         "m_{jj} [GeV/c^{2}]",
        'jj_pt':        "di-jet p_{T}  [GeV/c] ",
        'jj_dphi_signed': "#Delta#phi_{jj}  [rad] ",
        'w_mt':         "m_{T} (W) [GeV/c^{2}]",
        'wy_m':         "m_{W#gamma} [GeV/c^{2}]",
        'wy_pt':        "W#gamma p_{T}  [GeV/c]",
        'dphi_lepy':    'signed #Delta#phi_{Lepton #gamma}  [rad] ',
        'dphi_wy':      'signed #Delta#phi_{W#gamma}  [rad]',
        'dr_lepy':      '#Delta R_{Lepton #gamma}  [rad]',
        'dr_wy':        '#Delta R_{W#gamma}  [rad]'
        }

unit_dict['lep_pt']

'Lepton p_{T} [GeV/c] '

In [ ]:
# function of method 1
def plot_operator(operator, df_all, reconst_branch=reconst_branch[1:], set_log=False):
    # load values of coupling
    c1, c2, c3 = 1,10,100
    if operator[0] == 'T':
        c1, c2, c3 = 0.1,1,10

    # prepare bins for hist
    nbins = 5
    df = df_all.copy()
    c1_bin_size = df['c1_w'].sum() / 5
    c2_bin_size = df['c2_w'].sum() / 5
    c3_bin_size = df['c3_w'].sum() / 5
    c1_index = [0, c1_bin_size, c1_bin_size*2, c1_bin_size*3, c1_bin_size*4, c1_bin_size*5]
    c2_index = [0, c2_bin_size, c2_bin_size*2, c2_bin_size*3, c2_bin_size*4, c2_bin_size*5]
    c3_index = [0, c3_bin_size, c3_bin_size*2, c3_bin_size*3, c3_bin_size*4, c3_bin_size*5]

    # prepare canvas
    rows = 4
    if len(reconst_branch) < 16:
        rows = len(reconst_branch)**0.5
    cols = int(np.ceil(len(reconst_branch)/4))
    can = ROOT.TCanvas('can1', operator+'_can', rows*1000,750*cols)
    can.Divide(rows, cols)
    ican = 1

    hss=[]
    
    for r in reconst_branch:
        can.cd(ican)
        setlogx = lambda x: (x[-1] in ['e','t','m']) and (x[1] != 'j')
        can.cd(ican).SetLogx(setlogx(r))
        can.cd(ican).SetLogy(set_log)
        # def bins
        if r=='jj_pt':
            bins_c1 = [0,55,80,125,180,270,840]
            bins_c2 = bins_c1
            bins_c3 = bins_c1
        elif r=='jj_m':
            bins_c1 = [1000,1450,1800,2300,5300]
            bins_c2 = bins_c1
            bins_c3 = bins_c1
        elif r=='jj_dphi_signed':
            bins_c1 = [-3.1416, -2.9452,-2.7489,-2.4435, 0.0, 2.4435, 2.7489, 2.9452, 3.1416]
            bins_c2 = bins_c1
            bins_c3 = bins_c1
        else:
            df = df.sort_values(r)
            df['cum_c1'] = df['c1_w'].cumsum()
            df['cum_c2'] = df['c2_w'].cumsum()
            df['cum_c3'] = df['c3_w'].cumsum()
            bins_c1, bins_c2, bins_c3 = [df[r][0]], [df[r][0]], [df[r][0]]
            for i in range(1,5):
                bins_c1 += [ df[df['cum_c1']>=c1_index[i]][r][0] ]
                bins_c2 += [ df[df['cum_c2']>=c2_index[i]][r][0] ]
                bins_c3 += [ df[df['cum_c3']>=c3_index[i]][r][0] ]
            bins_c1 += [df[r].iloc[-1]]
            bins_c2 += [df[r].iloc[-1]]
            bins_c3 += [df[r].iloc[-1]]

        # generate hist
        h1 = ROOT.TH1F(r+"_h1", 'h1;'+r+';numbers', (len(bins_c1)-1), array('d',bins_c1))
        h2 = ROOT.TH1F(r+"_h2", 'h2;'+r+';numbers', (len(bins_c2)-1), array('d',bins_c2))
        h3 = ROOT.TH1F(r+"_h3", 'h3;'+r+';numbers', (len(bins_c3)-1), array('d',bins_c3))

        for i in range(len(df)):
            h1.Fill(df[r][i], df['c1_w'][i])
            h2.Fill(df[r][i], df['c2_w'][i])
            h3.Fill(df[r][i], df['c3_w'][i])

        h3.SetFillColorAlpha(800, 0.7)
        h2.SetFillColorAlpha(860+1,0.7)
        h1.SetFillColorAlpha(1,0.7)
        h1.SetFillStyle(3944)

        #h1.SetLineStyle(3)
        h3.SetLineColor(800-3)
        h2.SetLineColor(860-3)
        h1.SetLineColorAlpha(1, 0.6)

        h3.SetLineWidth(1)
        h2.SetLineWidth(1)
        h1.SetLineWidth(1)

        h3.SetMarkerStyle(9)
        h2.SetMarkerStyle(9)
        h1.SetMarkerStyle(9)

        hs = ROOT.THStack (r+"hs"," THStack ")
        hs.Add (h3)
        hs.Add (h2)
        hs.Add (h1)
        
        hs.Draw ("NOSTACK, L,h")
        if (r[1] != 'j'):
            hs.SetMinimum(1600)
        hss += [hs,h1,h2,h3]

        legend = ROOT.TLegend(0.75,0.75,0.99,0.99)
        legend.AddEntry ( h1 ,operator +' = '+str(c1)+' TeV^(-4)')
        legend.AddEntry ( h2 ,operator +' = '+str(c2)+' TeV^(-4)')
        legend.AddEntry ( h3 ,operator +' = '+str(c3)+' TeV^(-4)')
        legend.Draw (" same ")
        hss += [legend]

        hs.GetYaxis().SetTitle("Number")
        hs.GetXaxis().SetTitle(r)
        '''hs.GetYaxis().SetTitleOffset (0.8)
        hs.GetYaxis().SetLabelSize(0.02)
        '''
        can.cd(ican).SetBottomMargin (0.14)
        can.cd(ican).SetTopMargin (0)
        can.cd(ican).SetRightMargin(0)
        can.cd(ican).SetLeftMargin(0.14)
        ican += 1

        f = ROOT.TFile("histogram.root","recreate")
        f.cd()
        h1.Write()
        h2.Write()
        h3.Write()
        f.Close()
    
    can.Draw()
    can.Update()
    can.Print('test.pdf')
    #can.print ('test.png')
    can.SaveAs(generate_output_path(operator, add=reconst_branch[0], format='png'))
    can.Clear()


In [248]:
# function of method 1
def plot_operator2(operator, df_all, reconst_branch=reconst_branch[1:], long_format=1):
    # load values of coupling
    c1, c2, c3 = 1,10,100
    if operator[0] == 'T':
        c1, c2, c3 = 0.1,1,10

    # prepare bins for hist
    df = df_all.copy()

    # prepare canvas
    rows = 4
    if len(reconst_branch) < 16:
        rows = int(len(reconst_branch)**0.5)
    cols = int(np.ceil(len(reconst_branch)/rows))
    if not long_format:
        rows, cols = cols, rows 
    can = ROOT.TCanvas('can1', operator+'_can', rows*1000,750*cols)
    can.Divide(rows, cols)
    ican = 1

    hss=[]
    bins_arr = []
    
    for r in reconst_branch:
        set_log = r[-1] in ['e','t','m'] and r[1] != 'j'
        can.cd(ican)

        # upper part
        pad1 = ROOT.TPad (" pad1 "," pad1 " ,0,0,1,1)
        pad1.Draw()
        pad1.SetLogy ( set_log )
        pad1.cd()

        # def bins
        max_x = sm_df[r].max()
        min_x = sm_df[r].min()
        is_jj = (r in ['jj_pt', 'jj_m', 'jj_dphi_signed'])

        if is_jj:
            if r=='jj_pt':
                bins = [0,55,80,125,180,270,840]
            elif r=='jj_m':
                bins = [1000,1450,1800,2300,5300]
            elif r=='jj_dphi_signed':
                bins = [-3.1416, -2.9452,-2.7489,-2.4435, 0.0, 2.4435, 2.7489, 2.9452, 3.1416]
            h_sm = ROOT.TH1F(r+"_sm", 'sm;'+r+';numbers', (len(bins)-1), array('d',bins))
            h1 = ROOT.TH1F(r+"_h1", 'h1;'+r+';numbers', (len(bins)-1), array('d',bins))
            h2 = ROOT.TH1F(r+"_h2", 'h2;'+r+';numbers', (len(bins)-1), array('d',bins))
            max_x = bins[-1]
            min_x = bins[0]
        else:
            if set_log:
                bins = np.linspace(1.7, 1.7, 6).cumprod()
                bins = np.insert(bins, 0, 0)
                bins = [i/bins[-1]*max_x for i in bins]
                '''for i in range(8):
                    bins = np.append(bins, bins[-1]*1.7)'''
                bins = array('d',bins)  #[i for i in bins if i < max_x])
                h_sm = ROOT.TH1F(r+"_sm", 'sm;'+r+';numbers', len(bins)-1, bins)
                h1 = ROOT.TH1F(r+"_h1", 'h1;'+r+';numbers', len(bins)-1, bins)
                h2 = ROOT.TH1F(r+"_h2", 'h2;'+r+';numbers', len(bins)-1, bins)
            else:
                h_sm = ROOT.TH1F(r+"_sm", 'sm;'+r+';numbers', 8, min_x, max_x)
                h1 = ROOT.TH1F(r+"_h1", 'h1;'+r+';numbers', 8, min_x, max_x)
                h2 = ROOT.TH1F(r+"_h2", 'h2;'+r+';numbers', 8, min_x, max_x)
                bins = np.linspace(min_x,max_x,8+1)

        bins_arr += [bins]

        for i in range(len(sm_df)):
            h_sm.Fill(sm_df[r][i], sm_df['weight'][i])
        for i in range(len(df)):
            h1.Fill(df[r][i], df['c1_w'][i])
            h2.Fill(df[r][i], df['c2_w'][i])

        h_sm.SetFillColorAlpha(18,0.9)
        h1.SetFillColorAlpha(800,0)
        h2.SetFillColorAlpha(800,0)   
        h_sm.SetLineColorAlpha(392,0)
        h1.SetLineColorAlpha(38, 1)
        h2.SetLineColorAlpha(46, 1)
        h_sm.SetMarkerStyle(20)
        h1.SetMarkerStyle(21)
        h2.SetMarkerStyle(22)
        h1.SetMarkerSize(3)
        h2.SetMarkerSize(3)
        h1.SetMarkerColor(38)
        h2.SetMarkerColor(46)
        h_sm.SetMarkerColorAlpha(800,0)
        h2.SetLineWidth(1)
        h1.SetLineWidth(1)

        hs = ROOT.THStack (r+"hs"," THStack ")
        hs.Add (h_sm)
        hs.Add (h1)
        hs.Add (h2)
        
        hs.Draw ("NOSTACK, h")
        hs.SetMinimum(0.1)
        if not(r in ['j1_pt','wy_m', 'w_mt']):
            hs.SetMinimum(1)
        '''n, m = len(bins)-1, 0
        while (n >= 1) and (h2.GetBinContent(n) <= 0.5):
            n -= 1
        max_x = bins[n]
        while (m < len(bins)-1) and (h2.GetBinContent(n) <= 0.5):
            m += 1
        min_x = bins[m]'''
        hs.GetXaxis().SetRangeUser(min_x, max_x) 
        if r=='jj_pt':
            hs.SetMinimum(70)
        elif r=='jj_m':
            hs.SetMinimum(100)
        hss += [hs,h1,h2,h_sm]

        legend = ROOT.TLegend(0.65,0.75,0.99,0.99)
        legend.AddEntry(h_sm, "Standard Model", "f")
        legend.AddEntry ( h1 ,'f_{0} /#Lambda^{3} = {1} TeV^{2}'.format({operator}, c1,{-4},{4}),"lp")
        legend.AddEntry ( h2 ,'f_{0} /#Lambda^{3} = {1} TeV^{2}'.format({operator}, c2,{-4},{4}),"lp")
        legend.Draw (" same ")
        legend.SetFillColorAlpha(0, 0.8)
        hss += [legend]

        hs.GetYaxis().SetTitle("Number")
        hs.GetXaxis().SetTitle(unit_dict[r])
        pad1.SetBottomMargin (0.25)
        pad1.SetTopMargin (0)
        pad1.SetRightMargin(0)
        pad1.SetLeftMargin(0.13)


        # lower part
        pad2 = ROOT.TPad (" pad2 "," pad2 " ,0 ,0 ,1 ,0.25)
        pad2.Draw()
        pad2.cd()
        
        h1_low = h1-h_sm
        h1_low.Divide(h_sm)
        h1_low.Draw()

        h2_low = h2-h_sm
        h2_low.Divide(h_sm)
        h2_low.Draw('same')

        h1_low.GetYaxis().SetRangeUser(-0.2,0.2)
        h1_low.GetXaxis().SetRangeUser(min_x, max_x) 
        h1_low.GetYaxis().SetNdivisions (207)
        h1_low.SetMarkerSize(2)
        h2_low.SetMarkerSize(2)

        h1_markers, h2_markers = [], []
        for i in range(len(bins)):
            if (h1_low.GetBinContent(i) > 0.2):
                h1_markers += [ROOT.TMarker(h1_low.GetBinCenter(i), 0.2,21)]
            elif (h1_low.GetBinContent(i) < -0.2):
                h1_markers += [ROOT.TMarker(h1_low.GetBinCenter(i), -0.2,21)]
            if (h2_low.GetBinContent(i) > 0.2):
                h2_markers += [ROOT.TMarker(h2_low.GetBinCenter(i), 0.2,22)]
            elif (h2_low.GetBinContent(i) < -0.2):
                H2_markers += [ROOT.TMarker(h2_low.GetBinCenter(i), -0.2,23)] 
        for j in h1_markers:
            j.SetMarkerSize(2)
            j.SetMarkerColor(38)
            j.Draw('same')
        for j in h2_markers:
            j.SetMarkerSize(2)
            j.SetMarkerColor(46)
            j.Draw('same')
        hss += [h1_markers, h2_markers]
        #print(markers, len(markers))

        '''if set_log and r!='jj_m':
            min_x = 0
        elif r == 'jj_m':
            min_x =1000
        elif 'dphi' in r:
            min_x = -3.1'''
        line = ROOT.TLine (min_x, 0, max_x+1000,0)
        line.SetLineColor (1)
        line.SetLineWidth (1)
        line.Draw (" same ")
        hss += [h1_low,h2_low,line]
        
        h1_low.GetYaxis().SetTitle ("ALL/SM-1  ")
        h1_low.GetXaxis().SetTitle(unit_dict[r])
        h1_low.GetXaxis().SetTitleSize (0.25)
        h1_low.GetYaxis().SetTitleSize (0.14)
        h1_low.GetXaxis().SetLabelSize(0.15)
        h1_low.GetYaxis().SetLabelSize(0.14)
        h1_low.GetYaxis().SetTitleOffset (0.27)
        h1_low.GetXaxis().SetTitleOffset (0.7)
        pad2.SetTopMargin (0)
        pad2.SetRightMargin (0)
        pad2.SetLeftMargin(0.13)
        pad2.SetBottomMargin (0.45)

        ican += 1

        '''f = ROOT.TFile("histogram.root","recreate")
        f.cd()
        h1.Write()
        h2.Write()
        f.Close()'''
    
    can.Draw()
    can.Update()
    can.Print('test.pdf')
    #can.Print('test.png')
    #can.SaveAs(generate_output_path(operator))
    can.Clear()

    return bins_arr

In [276]:
#tmp = plot_operator2('T5', dfs_t[3], reconst_branch=l2, long_format=0)
#tmp = plot_operator2('M2', dfs_m[2], reconst_branch=l1,long_format=0)
#tmp = plot_operator2('M2', dfs_m[2], reconst_branch=['lep_pt','gam_e','wy_m','wy_pt'],long_format=0)
# tmp = plot_operator2('T6', dfs_t[4],reconst_branch=['dphi_lepy','dphi_wy', 'dr_lepy','dr_wy'],long_format=0)
tmp = plot_operator2('T6', dfs_t[4], reconst_branch=['lep_pt','gam_e','wy_m','dphi_lepy','dr_lepy'],long_format=0)

Warning in <TROOT::Append>: Replacing existing TH1: lep_pt_sm (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: lep_pt_h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: lep_pt_h2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: gam_e_sm (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: gam_e_h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: gam_e_h2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: wy_m_sm (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: wy_m_h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: wy_m_h2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: dphi_lepy_sm (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: dphi_lepy_h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing

In [226]:
l1 = ['gam_e','lep_pt','j0_pt','j1_pt','w_mt','wy_m','wy_pt']
l2 = ['jj_m','jj_pt','jj_dphi_signed']
l3 = ['dphi_lepy','dphi_wy','dr_lepy','dr_wy']

for i in range(len(operators_m)):
    tmp = plot_operator2(operators_m[i], dfs_m[i], reconst_branch=l2+l1+l3)
for i in range(len(operators_t)):
    tmp = plot_operator2(operators_t[i], dfs_t[i], reconst_branch=l2+l1+l3)




Warning in <TCanvas::Constructor>: Deleting canvas with same name: can1
Warning in <TROOT::Append>: Replacing existing TH1: jj_m_sm (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: jj_m_h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: jj_m_h2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: jj_pt_sm (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: jj_pt_h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: jj_pt_h2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: jj_dphi_signed_sm (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: jj_dphi_signed_h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: jj_dphi_signed_h2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: gam_e_sm (Potential memory leak).
Warning in <TROOT::Append>: Replacing existi

In [15]:
def plot_2_branch_rank(operator, branches, bins, df, weight_name = 'weight', c=1,isPDF=True):
    '''
    ranked by correlation
    '''
    df = df.copy()
    hs = []
    # add histogram
    for b in range(len(branches)):
        for a in range(b):    
            
            # variable name
            var_x = branches[a]
            var_y = branches[b]
            bins_x = bins[a]
            bins_y = bins[b]
 
            h1 = ROOT.TH2F(var_x+var_y, ";"+var_x+";"+var_y, len(bins_x)-1, array('d',bins_x), len(bins_y)-1, array('d',bins_y))

            '''if operator[0] == 'T':
                for i in range(len(df)):
                    h1.Fill(df[var_x][i], df[var_y][i], df['c2_w'][i])
            else:
                for i in range(len(df)):
                    h1.Fill(df[var_x][i], df[var_y][i], df['c1_w'][i])'''
            for i in range(len(df)):
                h1.Fill(df[var_x][i], df[var_y][i], df[weight_name][i])

            h1.Draw()
            hs += [h1]

    hs = sorted(hs, key=lambda x: x.GetCorrelationFactor(1,2), reverse=True)

    #ready to plot
    nvars = len(branches)   # number of variables to plot
    can = ROOT.TCanvas('can1', '',4000,4000)
    nplots = int(np.ceil(((nvars-1) * nvars/2) **0.5))
    can.Divide(nplots, nplots)

    ican = 1
    for h in hs:
        can.cd(ican).SetRightMargin(0)
        can.cd(ican).SetLeftMargin(0.15)
        can.cd(ican).SetTopMargin(0)
        can.cd(ican).SetBottomMargin(0.15)
        if (ican % nplots == 0) or (ican == len(hs)):
            can.cd(ican).SetRightMargin(.15)
 
        can.cd(ican).SetLogz(True)
        h.DrawClone('Colz')
        t = ROOT.TLatex()
        t.SetNDC()
        t.SetTextColor( 1 )
        t.SetTextSize( 0.05 )
        t.SetTextAlign( 12 )
        t.DrawLatex( 0.2, 0.95, 'correlation coefficient is %.3f'%h.GetCorrelationFactor(1,2) )
        ican += 1

    can.Update()
    can.Draw()
    #can.Print('test.png')

    can.SaveAs(generate_output_path(operator, add='2d'))


In [ ]:
#o = 'M2'

l1 = ['gam_e','lep_pt','wy_m','wy_pt']
l2 = ['jj_m','jj_pt','jj_dphi_signed']
vs = l2+l1
bins = []

for o in operators:
    df = generate_df(o)
    bins += plot_operator2(o, df, reconst_branch=vs)
    plot_2_branch_rank(o, vs, bins, df)

In [16]:
# function of method 1
def plot_operators(operators, dfs, reconst_branch=reconst_branch[1:], long_format=1):
    # load values of coupling
    c = 1
    if operators[0][0] == 'T':
        weight_col = 'c2_w'
    else:
        weight_col = 'c1_w'

    # prepare canvas
    rows = 4
    if len(reconst_branch) < 16:
        rows = int(len(reconst_branch)**0.5)
    cols = int(np.ceil(len(reconst_branch)/rows))
    if not long_format:
        rows, cols = cols, rows 
    can = ROOT.TCanvas('can1', operators[0][0] +'_can', rows*1000,1000*cols)
    can.Divide(rows, cols)
    ican = 1

    hss=[]
    colors = [2,3,4,6,7,8,9]
    markers = [20,21,22,23,33,34,47]
    
    for r in reconst_branch:
        can.cd(ican)
        can.cd(ican).SetLogy (1)

        # def bins
        max_x = sm_df[r].max()
        min_x = sm_df[r].min()
        is_jj = (r in ['jj_pt', 'jj_m', 'jj_dphi_signed'])

        if is_jj:
            if r=='jj_pt':
                bins = [0,55,80,125,180,270,840]
            elif r=='jj_m':
                bins = [1000,1450,1800,2300,5300]
            elif r=='jj_dphi_signed':
                bins = [-3.1416, -2.9452,-2.7489,-2.4435, 0.0, 2.4435, 2.7489, 2.9452, 3.1416]
            h_sm = ROOT.TH1F(r+"_sm", 'sm;'+r+';numbers', (len(bins)-1), array('d',bins))
        else:
            bins = np.linspace(1.7,1.7, 6).cumprod()
            bins = np.insert(bins, 0, 0)
            bins = [i/bins[-1]*max_x for i in bins]
            h_sm = ROOT.TH1F(r+"_sm", 'sm;'+r+';numbers', len(bins)-1, array('d',bins))

        for i in range(len(sm_df)):
            h_sm.Fill(sm_df[r][i], sm_df['weight'][i])

        h_sm.SetFillColorAlpha(18,0.9)
        h_sm.SetLineColorAlpha(392,0)
        h_sm.SetMarkerColorAlpha(800,0)

        hs = ROOT.THStack (r+"hs"," THStack ")
        hs.Add (h_sm)

        hos = []
        for j in range(len(operators)):
            operator = operators[j]
            ho = ROOT.TH1F(r+operator, operator+';'+r+';numbers', len(bins)-1, array('d',bins))

            df = dfs[j].copy()
            for i in range(len(df)):
                ho.Fill(df[r][i], df[weight_col][i])
    
            ho.SetFillColorAlpha(800,0)
            ho.SetLineColorAlpha(colors[j], 1)
            ho.SetMarkerColor(colors[j])
            ho.SetMarkerStyle(markers[j])
            ho.SetMarkerSize(3)
            ho.SetLineWidth(1)

            hs.Add (ho)
            hos += [ho]

        legend = ROOT.TLegend(0.65,0.65,0.99,0.99)
        legend.AddEntry(h_sm, "Standard Model", "f")
        for i in range(len(hos)):
            legend.AddEntry (hos[i],'f_{0} /#Lambda^{3} = {1} TeV^{2}'.format({operators[i]}, c,{-4},{4}),"lp")

        hs.Draw ("NOSTACK, h")
        legend.Draw (" same ")
        hs.SetMinimum(0.1)
        if not(r in ['j1_pt','wy_m', 'w_mt']):
            hs.SetMinimum(1)
        if r=='jj_pt':
            hs.SetMinimum(50)
        elif r=='jj_m':
            hs.SetMinimum(100)

        hss += [legend, hs, h_sm, hos]

        hs.GetYaxis().SetTitle("Number")
        hs.GetXaxis().SetTitle(unit_dict[r])
        can.cd(ican).SetBottomMargin (0.14)
        can.cd(ican).SetTopMargin (0)
        can.cd(ican).SetRightMargin(0)
        can.cd(ican).SetLeftMargin(0.125)

        ican += 1
    
    can.Draw()
    can.Update()
    can.Print('test.pdf')
    #can.Print('test.png')
    can.SaveAs(generate_output_path(operators[0][0] +'all'))
    can.Clear()



In [ ]:
plot_operators(operators_t, dfs_t, reconst_branch=['wy_m'], long_format=0)
plot_operators(operators_m, dfs_m, reconst_branch=['wy_m'], long_format=0)


In [87]:
tmp=[array('d', [0.0, 61.02, 103.73, 176.35, 299.79,369.75,509.65, 866.41]),
 array('d', [0.0, 141.90, 241.23, 410.09, 697.15, 1185.16, 2014.77]),
 array('d', [0.0, 184.31, 313.33, 532.66, 905.53, 1539.41, 2617.00]),
 array('d', [0.0, 69.56, 118.25, 201.03, 341.75, 580.98, 987.67])]

[array('d', [0.0, 61.021563263061026, 103.73665754720373, 176.35231783024634, 299.79894031141873, 509.65819852941183, 866.4189375000001]),
 array('d', [0.0, 141.90015648315537, 241.2302660213641, 410.09145223631896, 697.1554688017421, 1185.1642969629615, 2014.7793048370345]),
 array('d', [0.0, 184.31483813787406, 313.3352248343859, 532.669882218456, 905.538799771375, 1539.4159596113375, 2617.0071313392737]),
 array('d', [0.0, 69.56131269922787, 118.25423158868738, 201.03219370076852, 341.75472929130643, 580.9830397952209, 987.6711676518755])]

In [273]:
def plot_contribution(operators, dfs, branches, y_max=0.1, y_min=-0.1, col_width=0.17):

    can = ROOT.TCanvas('can1', 'can1', 3000, 2500)
    can.cd()
    pads = []
    hss = []
    bins = np.linspace(1.5, 1.5, 7).cumprod()
    bins = np.insert(bins, 0, 0)

    #def width
    row_width = 0.19
    left_space = 0.999-row_width*len(dfs)
    bottom_space = 1-col_width*len(branches)

    for i in range(len(dfs)):   #5
        operator = operators[i]
        df = dfs[i].copy()
        df_i = df.loc['i']
        df_q = df.loc['q']

        for j in range(len(branches)):  #4
            branch = branches[j]
            max_x = sm_df[branch].max()
            if max_x > 10:
                bins = [i/bins[-1]*max_x for i in bins]
            else:
                bins = np.linspace(sm_df[branch].min(), max_x, 7)
            bins = array('d',bins)
            can.cd()

            if j!=0 and i!=0:
                pad = ROOT.TPad ("pad"+str(i)+str(j),"pad"+str(i)+str(j) ,j*col_width+bottom_space, i*row_width+left_space, (j+1)*col_width+bottom_space, (i+1)*(row_width)+left_space)
            else:
                if i == 0 and j ==0:
                    pad = ROOT.TPad ("pad"+str(i)+str(j),"pad"+str(i)+str(j) ,j*col_width, i*row_width, (j+1)*col_width+bottom_space, (i+1)*(row_width)+left_space)
                elif i==0:
                    pad = ROOT.TPad ("pad"+str(i)+str(j),"pad"+str(i)+str(j) ,j*col_width+bottom_space, i*row_width, (j+1)*col_width+bottom_space, (i+1)*(row_width)+left_space)
                else:
                    pad = ROOT.TPad ("pad"+str(i)+str(j),"pad"+str(i)+str(j) ,j*col_width, i*row_width+left_space, (j+1)*col_width+bottom_space, (i+1)*(row_width)+left_space)
            pads += [pad]
            pad.Draw()
            pad.cd()

            h_i = ROOT.TH1F("i"+str(i)+str(j), 'hi;'+branch+';numbers    .', len(bins)-1, bins)
            h_q = ROOT.TH1F("q"+str(i)+str(j), 'hq;'+branch+';numbers    .', len(bins)-1, bins)
            h_a = ROOT.TH1F("a"+str(i)+str(j), 'ha;'+branch+';numbers    .', len(bins)-1, bins)
            h_s = ROOT.TH1F("s"+str(i)+str(j), 'hs;'+branch+';numbers    .', len(bins)-1, bins)

            for a in range(len(sm_df)):
                h_s.Fill(sm_df[branch][a], sm_df['weight'][a])
            for a in range(len(df_i)):
                h_i.Fill(df_i[branch][a], df_i['c1_w'][a])
                h_a.Fill(df_i[branch][a], df_i['c1_w'][a])
            for a in range(len(df_q)):
                h_q.Fill(df_q[branch][a], df_q['c1_w'][a])
                h_a.Fill(df_q[branch][a], df_q['c1_w'][a])

            h_i.Divide(h_s)
            h_q.Divide(h_s)
            h_a.Divide(h_s)
            hs = ROOT.THStack (branch+operator+"hs"," THStack ")
            hs.Add (h_i)
            hs.Add (h_q)
            hs.Add (h_a)
            hs.Draw ("NOSTACK, hist")

            hs.SetMinimum(y_min)
            hs.SetMaximum(y_max)
            hs.GetYaxis().SetNdivisions (407)
            hs.GetXaxis().SetNdivisions (407)
            hs.GetYaxis().SetTickLength(0.02)
            hs.GetXaxis().SetTickLength(0.02)
            hs.GetXaxis().SetTitle(unit_dict[branch]+' ')
            hs.GetXaxis().SetTitleOffset (0.85)
            hs.GetXaxis().SetTitleSize (0.115)
            hs.GetYaxis().SetTitle('Ratio to SM   ')
            hs.GetYaxis().SetTitleOffset (1)
            hs.GetYaxis().SetTitleSize (0.115)
            hs.GetXaxis().SetLabelSize (0.09)
            hs.GetYaxis().SetLabelSize (0.11)

            h_i.SetLineColorAlpha(38,1)
            h_a.SetLineColorAlpha(ROOT.kBlack,.8)
            h_q.SetLineColorAlpha(46, 1)
            h_i.SetLineStyle(ROOT.kDashed)
            h_q.SetLineStyle(ROOT.kDotted)
            h_a.SetLineWidth(1)
            h_i.SetMarkerSize(0)
            h_q.SetMarkerSize(0)
            h_a.SetMarkerSize(0)
            h_i.SetMarkerColorAlpha(800,0)
            h_a.SetMarkerColorAlpha(800,0)
            h_q.SetMarkerColorAlpha(800,0)

            t = ROOT.TLatex()
            t.SetNDC()
            t.SetTextColor( 1 )
            t.SetTextSize( 0.13 )
            t.SetTextAlign( 12 )
            if operator == 'M2':
                t.SetTextSize( 0.11 )
            t.DrawLatex( 0.85, 0.85, '#scale[1]{'+operator+'}')

            hss += [h_i, h_q, h_a, hs]
            pad.RedrawAxis('g')
            pad.SetBottomMargin (0)
            pad.SetTopMargin (0)
            pad.SetRightMargin(0)
            pad.SetLeftMargin(0)
            if j == 0:
                pad.SetLeftMargin(0.25)
            elif j == len(branches)-1:
                c=0.1
                if operator[0]=='M':
                    c=1

                if i == len(dfs)-1:
                    legend = ROOT.TLegend(0.02,0.45,0.8,0.97)
                    legend.AddEntry ( h_i ,'O_{0}'.format({'Interference term'}),"lp")
                    legend.AddEntry ( h_q ,'O_{0}'.format({'Quadratic term'}),"lp")
                    legend.AddEntry ( h_a ,'O_{0}'.format({'Total aQGC contribution'}),"lp")
                else:
                    legend = ROOT.TLegend(0.02,0.8,0.8,0.9)
                legend.SetHeader('f_{0} /#Lambda^{3} = {1} TeV^{2}'.format({operator}, c,{-4},{4}),"lp")
                legend.Draw (" same ")
                legend.SetFillColorAlpha(0, 0.8)
                hss += [legend]
            if i == 0:
                hs.GetYaxis().SetTitleSize (0.095)
                hs.GetYaxis().SetLabelSize (0.085)
                hs.GetYaxis().SetTitleOffset (1.2)
                hss+=[hs]
                pad.SetBottomMargin (0.25)
    
    can.Draw()
    can.Update()
    can.Print(branches[0]+'test.pdf') 




In [274]:
os = ['M2', 'T6', 'T1', 'T5', 'T0']
df_arr = [dfs_m[2], dfs_t[4], dfs_t[1], dfs_t[3], dfs_t[0]]
bs1 = ['lep_pt','gam_e','wy_m', 'jj_pt']
bs2 =['jj_m','jj_dphi_signed','dphi_lepy','dr_lepy']

plot_contribution(os,df_arr,bs1,y_max=0.5, y_min=-0.15, col_width=0.23)
print('complete first')
plot_contribution(os,df_arr,bs2,y_max=0.005, y_min=-0.0025, col_width=0.23)

complete first


Warning in <TROOT::Append>: Replacing existing TH1: i00 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: q00 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: a00 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: i01 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: q01 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: a01 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: i02 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: q02 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: a02 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: i03 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: q03 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: a03 (Potential memory leak).
Warning in <TROOT::Append>: 